## DataSet

In [1]:
from dataloader import get_dataloader
import config as CFG
from tokenizer import get_tokenizer

## Model

In [2]:
from models import CLIPModel
from losses import CLIPLoss

## Not new

In [3]:
from dataset import get_dataset
import os
import config as CFG
import torch
from torch.utils.data import DataLoader
import torchvision.transforms as transforms

In [4]:
transform_train = transforms.Compose([
            transforms.RandomCrop(224,pad_if_needed=True),
            transforms.RandomHorizontalFlip(), 
            transforms.ToTensor(), 
            transforms.Normalize((0.444, 0.421, 0.385), 
                                 (0.285, 0.277, 0.286))])

In [5]:
tokenizer = get_tokenizer(CFG.text_model_name)
dataset = get_dataset(tokenizer=tokenizer,transform=transform_train,split="train")

Using downloaded and verified file: ./flickr30k/flickr30k_train.json


In [8]:
dataset.__getitem__(26)

{'image': tensor([[[-1.4203, -1.2965, -1.1451,  ...,  1.5381,  1.6344,  1.5931],
          [-0.9800, -1.3377, -1.5304,  ...,  1.3317,  1.5106,  1.5106],
          [-0.9249, -1.4203, -1.2139,  ...,  1.4418,  1.4418,  1.5381],
          ...,
          [-1.0350, -1.2414, -1.3790,  ..., -0.7736, -0.3195, -0.1131],
          [-1.2689, -1.0901, -1.1313,  ..., -0.3883, -0.3608, -0.2369],
          [-1.3377, -1.2552, -1.3928,  ..., -0.0993, -0.0305, -0.1269]],
 
         [[-1.3358, -1.3783, -1.2792,  ...,  1.4956,  1.5664,  1.5240],
          [-1.0951, -1.1801, -1.5199,  ...,  1.2550,  1.3965,  1.3965],
          [-1.1093, -1.2933, -1.1235,  ...,  1.3116,  1.3116,  1.3965],
          ...,
          [-1.0244, -1.2650, -1.4208,  ..., -0.6846, -0.3023, -0.1183],
          [-1.1659, -1.2084, -1.1235,  ..., -0.3590, -0.3873, -0.3873],
          [-1.2367, -1.3358, -1.3500,  ..., -0.2457, -0.1466, -0.2882]],
 
         [[-1.3187, -1.3050, -1.3462,  ...,  1.0534,  1.1083,  1.0671],
          [-1.2639,

In [15]:
dataset.img_ids

{0: {'image': 'flickr30k-images/1000092795.jpg',
  'caption': ['Two young guys with shaggy hair look at their hands while hanging out in the yard.',
   'Two young, White males are outside near many bushes.',
   'Two men in green shirts are standing in a yard.',
   'A man in a blue shirt standing in a garden.',
   'Two friends enjoy time spent together.'],
  'image_id': 0},
 1: {'image': 'flickr30k-images/10002456.jpg',
  'caption': ['Several men in hard hats are operating a giant pulley system.',
   'Workers look down from up above on a piece of equipment.',
   'Two men working on a machine wearing hard hats.',
   'Four men on top of a tall structure.',
   'Three men on a large rig.'],
  'image_id': 1},
 2: {'image': 'flickr30k-images/1000268201.jpg',
  'caption': ['A child in a pink dress is climbing up a set of stairs in an entry way.',
   'A little girl in a pink dress going into a wooden cabin.',
   'A little girl climbing the stairs to her playhouse.',
   'A little girl climbing i

## DDP

In [7]:
from torchvision.datasets.utils import download_url
from torch.utils.data import  Dataset
import json
from dataset import pre_caption
import config as CFG
from torchvision.datasets import Flickr30k
import random

In [38]:

class flickr30k_test(Dataset):
    def __init__(self, tokenizer,transform,image_root, ann_root, split, max_words=CFG.max_length, prompt=CFG.prompt):        
        '''
        image_root (string): Root directory of images (e.g. data/)
        ann_root (string): directory to store the annotation file
        split (string): one of "train" or "test"
        '''        
        train = 'https://storage.googleapis.com/sfr-vision-language-research/datasets/flickr30k_train.json'
        test = 'https://storage.googleapis.com/sfr-vision-language-research/datasets/flickr30k_test.json'

        self.tokenizer = tokenizer

        self.split = split
        assert self.split in ("train","test")

        if self.split == "train":
            url = train
            filename = 'flickr30k_train.json'
        else:
            url = test
            filename = 'flickr30k_test.json'

        download_url(url,ann_root)
        
        self.annotation = json.load(open(os.path.join(ann_root,filename),'r'))
        self.transform = transform
        self.image_root = image_root
        self.max_words = max_words      
        self.prompt = prompt
        
        self.img_ids = {} 

        for ann in self.annotation:
            img_id = ann['image_id']
            if img_id not in self.img_ids.keys():
                self.img_ids[img_id] = ann
                self.img_ids[img_id]["caption"] = [self.img_ids[img_id]["caption"]]
            
            else: 
                ls = self.img_ids[img_id]["caption"]
                ls.append(ann["caption"])               
                self.img_ids[img_id]["caption"] = ls

        self.annotation = None
        
    def __len__(self):
        return len(self.img_ids)
    
    def __getitem__(self, index):    
        
        item = self.img_ids[index]
        
        image_path = os.path.join(self.image_root,item['image'])        
        image = Image.open(image_path).convert('RGB')   
        image = self.transform(image)
        
        caption = self.prompt+pre_caption(random.choice(item['caption']), self.max_words)
        
        caption_encoded = self.tokenizer(caption,padding="max_length",max_length=self.max_words)

        return {"image" :image, "input_ids": torch.as_tensor(caption_encoded["input_ids"]), "attention_mask": torch.as_tensor(caption_encoded["attention_mask"])}

In [39]:
ds = flickr30k_test(tokenizer=tokenizer,transform=transform_train,image_root=CFG.image_root,ann_root=CFG.ann_root,split="train")

Using downloaded and verified file: ./flickr30k/flickr30k_train.json


In [40]:
ds.__getitem__(0)

NameError: name 'Image' is not defined

In [36]:
random.choice(ds.img_ids[0]["caption"])

'Two young, White males are outside near many bushes.'

In [31]:
from dataset import flickr30k

In [32]:
ds2 = flickr30k(tokenizer=tokenizer,transform=transform_train,image_root=CFG.image_root,ann_root=CFG.ann_root,split="train")

Using downloaded and verified file: ./flickr30k/flickr30k_train.json


In [25]:
ds2.annotation[:5]

[{'image': 'flickr30k-images/1000092795.jpg',
  'caption': 'Two young guys with shaggy hair look at their hands while hanging out in the yard.',
  'image_id': 0},
 {'image': 'flickr30k-images/1000092795.jpg',
  'caption': 'Two young, White males are outside near many bushes.',
  'image_id': 0},
 {'image': 'flickr30k-images/1000092795.jpg',
  'caption': 'Two men in green shirts are standing in a yard.',
  'image_id': 0},
 {'image': 'flickr30k-images/1000092795.jpg',
  'caption': 'A man in a blue shirt standing in a garden.',
  'image_id': 0},
 {'image': 'flickr30k-images/1000092795.jpg',
  'caption': 'Two friends enjoy time spent together.',
  'image_id': 0}]